In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

Using TensorFlow backend.


In [2]:
data = DataLoader(config_mixed_coco2014_coco2014)
# print(config_mixed_flickr8k_flickr8k)

In [3]:
data= preprocess_data(data)

Descriptions cleaned.
[['a', 'group', 'of', 'people', 'are', 'partying', 'at', 'a', 'masquerade', 'party'], ['a', 'lady', 'in', 'red', 'and', 'black', 'grins', 'at', 'the', 'camera', 'at', 'a', 'costume', 'party'], ['a', 'woman', 'in', 'a', 'domino', 'mask', 'and', 'severe', 'hair', 'is', 'at', 'a', 'party'], ['a', 'woman', 'in', 'a', 'red', 'dress', 'and', 'black', 'mask', 'is', 'on', 'a', 'crowded', 'dance', 'floor'], ['a', 'woman', 'wears', 'a', 'red', 'dress', 'and', 'a', 'black', 'mask', 'while', 'people', 'dance', 'behind', 'her']]
Descriptions wraped into start and stop words.
['START a child in a pink dress is climbing up a set of stairs in an entry way STOP', 'START a girl going into a wooden building STOP', 'START a little girl climbing into a wooden playhouse STOP', 'START a little girl climbing the stairs to her playhouse STOP', 'START a little girl in a pink dress going into a wooden cabin STOP']


2022-09-07 18:15:56.060520: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-07 18:15:56.061025: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


Encoded train images loaded from: 
./mixed_flickr8k_8k_n/Pickle/encoded_train_images.pkl
Encoded train images loaded from:  
./mixed_flickr8k_8k_n/Pickle/encoded_test_images.pkl
Number of training captions  30000
Description Length: 37
preprocessed words 7589 -> 1654
Vocab size:  1655
Found 400000 word vectors.


In [4]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, \
    Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.models import Model
from keras import Input, layers
from keras import callbacks
from eval_utils import calculate_results, prepare_for_evaluation
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array
import pickle

In [5]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    X1, X2, y = list(), list(), list()
    n = 0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n += 1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n = 0

class ModelImpl:
    def __init__(self, data):
        self.data=data
        inputs1 = Input(shape=(2048,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation='relu')(fe1)
        inputs2 = Input(shape=(self.data.max_length,))
        se1 = Embedding(self.data.vocab_size, general[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation='relu')(decoder1)
        outputs = Dense(self.data.vocab_size, activation='softmax')(decoder2)
        self.model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        self.model.summary()
        self.model.layers[2]

        self.model.layers[2].set_weights([self.data.embedding_matrix])
        self.model.layers[2].trainable = False

        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer())
        self.setup()

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 2
        self.number_pics_per_bath = 100
        self.steps = len(self.data.train_captions_wrapped) // self.number_pics_per_bath

    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            callback = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            generator = data_generator(self.data.train_captions_wrapped,
                                       self.data.encoded_images_train,
                                       self.data.wordtoix,
                                       self.data.max_length,
                                       self.number_pics_per_bath,
                                       self.data.vocab_size)
            self.model.fit(generator, epochs=self.epochs,
                           steps_per_epoch=self.steps,
                           callbacks=[callback],
                           verbose=1)
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoded_images_test, self.data, self.model)
        with open("expected.pkl","wb") as encoded_pickle:
            pickle.dump(expected, encoded_pickle)
        with open("results.pkl","wb") as encoded_pickle2:
            pickle.dump(results, encoded_pickle2)
#         with open("expected.pkl", 'rb') as encoded_pickle:
#             expected = pickle.load(encoded_pickle)
#         with open("results.pkl", 'rb') as encoded_pickle:
#             results = pickle.load(encoded_pickle)
#         out = calculate_results(expected, results, self.data.configuration)
        print(out)

model=ModelImpl(data)


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 37)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 37, 199)      329345      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [6]:
model.train()

In [7]:
model.evaluate()

tokenization...


PTBTokenizer tokenized 64178 tokens at 433215.77 tokens per second.
PTBTokenizer tokenized 20086 tokens at 227258.45 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 19087, 'reflen': 12415, 'guess': [19087, 18087, 17087, 16087], 'correct': [5658, 1846, 413, 115]}
ratio: 1.5374144180425664
Bleu_1: 0.296
Bleu_2: 0.174
Bleu_3: 0.090
Bleu_4: 0.048
computing METEOR score...
METEOR: 0.127
computing Rouge score...
ROUGE_L: 0.330
computing CIDEr score...
CIDEr: 0.131
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.5 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.0 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5 sec].
Loading classif

SPICE evaluation took: 1.420 min
SPICE: 0.075
computing WMD score...
WMD: 0.380


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

